In [ ]:
#www.linkedin.com/in/ibrahim-telli-2058a9220
#İZMİR BAKIRÇAY ÜNİVERSİTESİ EEM&BM Öğrencisi İBRAHİM TELLİ
#KODUN AMACI: Amerika BM'de Elektrikli Araç Sayısının Artışı üzerine olan bir veri seti üzerinde veri görselleştirme.
#DOSYA ÜZERİNDEKİ EKSİK KODLARI DOLDURMA-CSV DOSYASI ÜZERİNDEN VERİ OKUYARAK VERİ GRAFİĞİ ÇIKARMA-LİNEER REGRESYON KULLANIMI

In [ ]:
import pandas as pd # Bu kütüphane temel olarak zaman etiketli serileri ve sayısal tabloları işlemek için bir veri yapısı oluşturur 
import numpy as np # Çok boyutlu dizileri ve matrisleri destekleyen, bu diziler üzerinde çalışacak üst düzey matematiksel işlevler ekleyen bir kitaplıktır
import seaborn as sns #MatPlotLib kütüphanesini temel alan bir Python görselleştirme kütüphanesidir.
import matplotlib.pyplot as plt #Pyplot, Matplotlib'i kullanarak çizim oluşturmanızı sağlar. Bu şekilde, plt olarak kısaltılmış olan pyplot modülüne erişebilirsiniz.
import plotly.express as px #Veri görselleştirme ve dashboard oluşturmaya yarayan çeşitli fonksiyonlar içeren bir kütüphanedir

In [ ]:
df=pd.read_csv("C:/Users/elif/Desktop/Electric_Vehicle_Population_Data.csv") #CSV dosyasını okutup df değişkenine atama yapıldı

In [ ]:
df.info() #Veri seti hakkında bilgi alındı.
df.head() #Veri seti gösterildi.
df.drop(['Base MSRP','Legislative District','DOL Vehicle ID','Electric Utility','2020 Census Tract'],axis=1,inplace=True)
#Kullanılmayacak olan veriler drop kodu ile veriden silindi.

In [ ]:
df.isnull().sum() #Veri setideki toplam eksik bilgi gösterildi.

In [ ]:
df.Model.fillna(method="ffill",inplace=True) 
df.Location.fillna(method="ffill",inplace=True) #Veri setindeki boşluklar tam olarak bilinemeyeceği için- 
#veri kendi satırının bir üstündeki veri ile dolduruldu.

In [ ]:
df.isnull().sum() #Veri setideki toplam eksik bilgi setine tekrar bakıldı ve eksik veriler giderildiği gözlemlendi.

In [ ]:
# VERİ GRAFİKLERİ

In [ ]:
corr_matrix = df.corr()

# Korelasyon matrisini yazdırma
print(corr_matrix)

# Korelasyon matrisini görselleştirme
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Korelasyon Matrisi')
plt.show()

In [ ]:
car_market_share = pd.DataFrame(df.groupby('Make')['VIN'].count().reset_index()).sort_values(by='VIN',ascending=False).reset_index(drop=True)
car_market_share.columns = ['model','count']
car_market_share_top_5 = car_market_share.head(5)
car_market_share_bottom_5 = car_market_share.tail(5)

fig, axes = plt.subplots(1, 2, figsize=(20, 5))
sns.barplot(data=car_market_share_top_5, x="model", y="count",ax=axes[0])
sns.barplot(data=car_market_share_bottom_5, x="model", y="count",ax=axes[1])

axes[0].set_title('EN ÇOK OLAN 5 MARKA')
axes[1].set_title('EN AZ OLAN 5 MARKA') # BarPlot ile ülkedeki en çok araca sahip ilk 5 ve son 5 marka grafiği gösterildi.

In [ ]:
#Lokasyon bilgisine göre Araç Sayısı Grafiği

Counties = df.groupby('Location').count().sort_values(by='City',ascending=False)['City'].index
values = df.groupby('Location').count().sort_values(by='City',ascending=False)['City'].values

px.bar(x=list(Counties)[:10],y=values[:10],labels={'x':"Lokasyon",'y':"Araç Sayısı"},color=values[:10])

In [ ]:
#Posta Kodu bilgisine göre Top 10 Modein Araç Sayısı Grafiği
car_counts_pc = df['PostalCode'].value_counts().nlargest(10)
plt.bar(car_counts_pc.index, car_counts_pc.values)
plt.xlabel('Posta Kodu')
plt.ylabel('Araba Sayısı')
plt.title('Posta Koduna Göre İlk 10 Modelin Araba Sayısı')
plt.show()
car_counts_pc_df = car_counts_pc.to_frame()
car_counts_pc_df.style.background_gradient(cmap='Blues')

In [ ]:
#En çok araç bulunan ilk 10 ilçedeki Araç Sayısı Grafiği
Companies = df.groupby('County').count().sort_values(by='City',ascending=False)['City'].index
values = df.groupby('County').count().sort_values(by='City',ascending=False)['City'].values

plt.figure(figsize=(9,5))
sns.barplot(x=list(Companies)[:10],y=values[:10],edgecolor='.2',palette='Set2')
plt.xticks(rotation='90')
plt.xlabel('İlçeler')
plt.ylabel('Sayısı')
plt.show()

In [ ]:
#Eyaletlerdeki araçların toplam menzil sınırını gösteren grafik
plt.figure(figsize=(20, 10))
sns.violinplot(x='State', y='ElectricRange', data=df)
plt.title('Violin Grafiği')
plt.xlabel('Eyaletler')
plt.ylabel('Menzil')
plt.show()

In [ ]:
#En Çok Araç Bulunan İlk 10 Şehrin Sahip Olduğu İlk 10 Markanın Dağılımının Grafiği
cnt_MkCity = df.groupby(['City', 'Make']).size().reset_index(name='Count')
grouped_data_cty = cnt_MkCity.groupby(['City', 'Make'])['Count'].sum().reset_index()

city_counts = grouped_data_cty.groupby('City')['Count'].sum().reset_index()
make_counts = grouped_data_cty.groupby('Make')['Count'].sum().reset_index()

top_cities = city_counts.sort_values(by='Count', ascending=False).head(10)
top_makes = make_counts.sort_values(by='Count', ascending=False).head(10)

filtered_data_Cty = grouped_data_cty[grouped_data_cty['City'].isin(top_cities['City']) & grouped_data_cty['Make'].isin(top_makes['Make'])]

pivoted_data_cty = filtered_data_Cty.pivot(index='City', columns='Make',values='Count').fillna(0)

pivoted_data_cty.plot(kind='bar', stacked=True)

plt.title('En Çok Araç Bulunan İlk 10 Şehrin Sahip Olduğu İlk 10 Markanın Dağılımı')
plt.xlabel('Şehir')
plt.ylabel('Araba Sayısı')
plt.show()
pivoted_data_cty.head()

In [ ]:
#En Çok Araç Bulunan İlk 10 Eyaletin Sahip Olduğu İlk 10 Markanın Dağılımının Grafiği
cnt_Mk_St = df.groupby(['State', 'Make']).size().reset_index(name='Count')
grouped_data_St = cnt_Mk_St.groupby(['State', 'Make'])['Count'].sum().reset_index()
st_counts = grouped_data_St.groupby('State')['Count'].sum().reset_index()
make_counts = grouped_data_St.groupby('Make')['Count'].sum().reset_index()
top_States = st_counts.sort_values(by='Count', ascending=False).head(10)
top_makes = make_counts.sort_values(by='Count', ascending=False).head(10)

filtered_data_St = grouped_data_St[grouped_data_St['State'].isin(top_States['State']) & grouped_data_St['Make'].isin(top_makes['Make'])]
pivoted_data_St = filtered_data_St.pivot(index='State', columns='Make',values='Count').fillna(0)
pivoted_data_St.plot(kind='bar', stacked=True)

plt.title('En Çok Araç Bulunan İlk 10 Eyaletin Sahip Olduğu İlk 10 Markanın Dağılımı')
plt.yscale('log')
plt.xlabel('Eyalet')
plt.ylabel('Araba Sayısı')
plt.show()
pivoted_data_St.head(10)

In [ ]:
# En çok sahip olunan ilk 10 modelin sayısını gösteren grafik
Models = df.groupby('Model').count().sort_values(by='City',ascending=False)['City'].index
values = df.groupby('Model').count().sort_values(by='City',ascending=False)['City'].values


plt.figure(figsize=(9,5))
sns.barplot(x=list(Models)[:10],y=values[:10],edgecolor='.2',palette='Set2')
plt.xticks(rotation='90')
plt.xlabel('Modeli')
plt.ylabel('Sayısı')
plt.show()

In [ ]:
# Pasta grafiği ile CAFV kriterini sağlayan araçların oranı

eligibility_counts = df['(CAFV)Eligibility'].value_counts()

plt.figure(figsize=(8, 6))
plt.pie(eligibility_counts, labels=eligibility_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('(CAFV) Kriteri Sağlama Durumu')
plt.show()

In [ ]:
#İlk 10 markanın sattığı araçların elektrikli araç tipinin gösterildiği Pasta Grafiği
Companies=df['Make']
top_10_companies = list(Companies)[:10]
for i in top_10_companies:
    data = df[df['Make']==i]
    data = data.groupby('Model').count().sort_values(by='City',ascending=False).index
    print('Top selling model for',i,'is ----------->',data[0])

empty_series = pd.Series(dtype='object')
top_10_companies = list(Companies)[:10]
for index,i in enumerate(top_10_companies):
    data = df[df['Make']==i]
    labels = list(data.groupby('EVType').count()['City'].index)
    values = list(data.groupby('EVType').count()['City'].values)
    fig = px.pie(names=labels,values=values,width=700,height=400,title=str(i))
    fig.show()

In [ ]:
#Model yılına göre hangi araç markasının ne kadar sayıda aracı olduğunu gösteren grafik
top_10_vehicles = list(df.groupby('Model').count().sort_values(by='City',ascending=False)['City'].index)[:10]
top_10_range =  df.sort_values(by='ElectricRange',ascending=False)['Model'].unique()[:10]
data = df.copy()
data['top_10'] = data['Make'].apply(lambda x:1 if x in top_10_companies else 0)
data = data[data['top_10']==1]
data = data[data['ModelYear']>=2011]
plt.figure(figsize=(12,5))
sns.countplot(x = 'ModelYear',hue='Make',data=data)

In [ ]:
locations = list(df.groupby('Location').count()['County'].index) # Amerikada enlem-boylam ilişkisine göre araç sayısının nokta grafiği
values = list(df.groupby('Location').count()['County'].values)
Location_data = pd.DataFrame({'Locations':locations,'Count':values})
Location_data['Enlem'] = Location_data['Locations'].apply(lambda x:float(x.split(' ')[1][1:]))
Location_data['Boylam'] = Location_data['Locations'].apply(lambda x:float(x.split(' ')[2][:-1]))

plt.figure(figsize=(10,5))
plt.scatter(x = Location_data['Enlem'],y = Location_data['Boylam'],s=Location_data['Count'])
plt.xlim(-130,-60)
plt.ylim(20,60)

In [ ]:
df['Enlem'] = df['Location'].apply(lambda x:float(x.split(' ')[1][1:]))
df['Boylam'] = df['Location'].apply(lambda x:float(x.split(' ')[2][:-1])) #Genel enlem boylam ataması

In [ ]:
# Amerikada enlem-boylam ilişkisine göre CAFV ilişkinin nokta grafiği
plt.figure(figsize=(10,5))   
sns.scatterplot(x = 'Enlem',y = 'Boylam',data=df,hue='(CAFV)Eligibility',palette='Set2')
plt.xlim(-130,-60)
plt.ylim(20,50)

In [ ]:
# Amerikada enlem-boylam ilişkisine elektrikli araç tipinin nokta grafiği
plt.figure(figsize=(10,5))
sns.scatterplot(x = 'Enlem',y = 'Boylam',data=df,hue='EVType',palette='Set1')
plt.xlim(-130,-60)
plt.ylim(20,50)

In [ ]:
#Amerika haritası üzerinde eyaletlerin sahip olduğu araç sayısını gösteren coğrafya grafiği
states = list(df.groupby('State').count().sort_values(by='City',ascending=False)['City'].index)
values = df.groupby('State').count().sort_values(by='City',ascending=False)['City'].values
data = pd.DataFrame(df.groupby('State').count().sort_values(by='City',ascending=False)['City'])
data = data.reset_index()
data.columns = ['State','Count']
fig = px.choropleth(data,
                    locations='State', 
                    locationmode="USA-states", 
                    color='Count',
                    color_continuous_scale="greens", 
                    scope="usa")

fig.show() 

In [ ]:
#LİNEER REGRESYON MODELLEME

In [ ]:
#Model yıllarına göre elimizde bulunan toplam araç sayısı gösterildi
total_electric = df.pivot_table(columns=['ModelYear'], aggfunc='size')
print(total_electric)

In [ ]:
#NCars değişkenini tablo haline getiriyoruz.
Ncars = total_electric.reset_index()
Ncars = Ncars.rename(columns={"Model Year": "year", 0: "ToplamArabaSayısı"})
#2023 Modelleri atıyoruz çünkü artış modelini bozuyor.
Ncars = Ncars.drop([0, 19])

In [ ]:
x = Ncars["ModelYear"]
y = Ncars["ToplamArabaSayısı"]   #Grafik için satır sutün atıyoruz.

EskiAraçlar = Ncars.iloc[:6, :]  #Eski ve yeni model araçları incelemek için araç yıllarını 98-10 12-22 yılları olarak ayırıyoruz.
YeniAraçlar = Ncars.iloc[6:, :]

x1 = EskiAraçlar["ModelYear"]
y1 = EskiAraçlar["ToplamArabaSayısı"]

x2 = YeniAraçlar["ModelYear"]
y2 = YeniAraçlar["ToplamArabaSayısı"]


In [ ]:
#98-10 yılları arasında çıkmış araçların sayısının çizgi grafiği
fig, ax = plt.subplots()

ax.plot(x1, y1, linewidth=2.0)
plt.xlabel('Araba Sayısı')
plt.ylabel('Model Yılı')
plt.show()

In [ ]:
#12-22 yılları arasında çıkmış araçların sayısının çizgi grafiği
fig, ax = plt.subplots()

ax.plot(x2, y2, linewidth=2.0)
plt.xlabel('Araba Sayısı')
plt.ylabel('Model Yılı') 
plt.show()

In [ ]:
# Araba sayısının yıllara bağlı olarak artışını ele alan verinin lineer regresyon ile grafiklenmesi
sns.lmplot(x="ModelYear", 
           y="ToplamArabaSayısı",
          data = YeniAraçlar)
plt.xlabel('Araba Sayısı')
plt.ylabel('Model Yılı')

In [ ]:
# Buraya kadar geldiyseniz ve CSV dosyasına ulaşmak istiyorsanız LinkedIn üzerinden bana ulaşabilirsiniz.
#İYİ ÇALIŞMALAR...